In [ ]:
# | default_exp templates

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

import jupyter_black
import nbdev.showdoc as showdoc
from fastcore.test import *
from jinja2 import UndefinedError
from sal.utils import files

In [ ]:
# | hide
jupyter_black.load()

In [ ]:
# | exporti
from pathlib import Path
from typing import Any, Optional, Union
from jinja2 import (
    Environment,
    BaseLoader,
    Template,
    StrictUndefined,
    TemplateNotFound,
    ChoiceLoader,
    DictLoader,
    FileSystemLoader,
)

from sal.core import Data
from sal.frontmatter import FrontMatter

# Template relathed things

In [ ]:
# | exporti
# | hide
def _get_env() -> Environment:
    return Environment(loader=BaseLoader(), undefined=StrictUndefined)

    We need a rendering function capable of renderering a `template`, with `filters` and `context variables`...

In [ ]:
# | export
def render_to_remove(
    template: str,  # template in string form
    filters: Optional[dict] = None,  # jinja filters
    **kwargs: Any,
) -> str:
    if not filters:
        filters = {}

    env = _get_env()
    env.filters.update(filters)

    jinja: Template = env.from_string(template)
    result: str = jinja.render(**kwargs)

    return result

In [ ]:
showdoc.show_doc(render_to_remove)

In [ ]:
# | hide
filters = {"upper": lambda n: n.upper()}
template = 'this is a template is this is my name "{{ name | upper }}" in upper case'
kwargs = dict(name="mauro")

result = render_to_remove(template, filters=filters, **kwargs)
expected = 'this is a template is this is my name "MAURO" in upper case'
test_eq(result, expected)

try:
    render_to_remove("{{ name }}")
except UndefinedError:
    pass

    Let's wrap up the rendering function from the `core` into an usable class

In [ ]:
# | exporti
class TemplateRenderer:
    def render(self, template: Optional[str] = None, **kwargs: Any) -> str:
        if template is None:
            raise RuntimeError("Missing template")
        return render_to_remove(template, **kwargs)

## Template loading

We will need a way to get the templates

In [ ]:
# | exporti
MissingTemplateException = TemplateNotFound


class TemplateLoader:
    def __init__(
        self,
        templates: Optional[dict[str, str]] = None,
        folders: Optional[list[Path]] = None,
    ):
        self.frontmatter_handler = FrontMatter()
        loaders: list[Union[DictLoader, FileSystemLoader]] = [
            DictLoader(templates or {})
        ]
        if folders:
            for folder in folders:
                loaders.append(FileSystemLoader(folder))
        self.loader = ChoiceLoader(loaders)

    def get_source(self, name: str, frontmatter: Optional[bool] = False) -> str:
        if not name.endswith(".jinja2"):
            name = name + ".jinja2"
        template, _, _ = self.loader.get_source(_get_env(), name)
        if not frontmatter:
            return self.frontmatter_handler.get_content(template)
        return self.frontmatter_handler.get_frontmatter_source(template)

    def get_frontmatter_source(self, name: str) -> str:
        return self.get_source(name, frontmatter=True)

    @classmethod
    def from_directories(cls, directories: list[Path]) -> "TemplateLoader":
        return cls(folders=directories)

In [ ]:
# | hide

with files(
    {
        f"tmp/templates/model.jinja2": "my model template",
    }
):
    tpl_loader = TemplateLoader.from_directories(["tmp/templates"])
    tpl_loader.get_source("model")

## Tying rendering and loading together

And finally, put these 2 together to form a class to render a `Data` instance

In [ ]:
# | exporti
# TODO remove "any" typings
class Renderer:
    # if no template is passed in, we use the DEFAULT_TEMPLATE
    DEFAULT_TEMPLATE = "{% for child in children %}{{ child | render }}{% endfor %}"

    def __init__(
        self,
        *,
        renderer: TemplateRenderer,
        repository: TemplateLoader,
        filters: Optional[dict] = None
    ):
        self.renderer = renderer
        self.repository = repository
        self.filters = filters or {}

    def render(self, data: Data, template: Optional[str] = None) -> str:
        if template is None:
            template = self.repository.get_source(data.name)

        return self.renderer.render(
            template=template,
            **data.attrs,
            filters={**self.filters, "render": self.render},
            node=data,
            children=data.children,
        )

    def process(self, data: Data) -> str:
        return self.render(data)

    def get_source(self, *args: Any, **kwargs: Any) -> str:
        return self.repository.get_source(*args, **kwargs)

    def get_metadata_for_template(self, path: str, data: Data) -> dict:
        template = self.repository.get_frontmatter_source(path)

        rendered = self.render(data, template)

        ret: dict = self.repository.frontmatter_handler.parse(rendered)

        return ret

The entry point for this class, after `__init__`, is the `process` method

---

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()